## Time Series Exploration

In [1]:
# general imports
import pandas as pd
import requests
import numpy as np
import datetime
import acquire
import prepare
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# set figure size
plt.rcParams["figure.figsize"] = (8,6)

# specify decimal places to display
pd.set_option('display.float_format', lambda x: '%.2f' % x)

Do your work for this exercise in a notebook named explore. Use the techniques in the lesson to explore the store item demand dataset and the opsd dataset.

For the store item demand data, you will need to choose a method of aggregating such that each observation is a unique date. For both datasets you should choose a method of splitting your data and only explore the training split.

## Store Data Set

In [2]:
df = acquire.get_groceries()

In [3]:
df = prepare.prep_groceries(df, 'sale_date')

In [ ]:
print('Date Range:', df.index.min(), 'to', df.index.max())
print('Shape:', df.shape)

In [ ]:
df.head()

In [ ]:
df.sales_total.plot()